In [ ]:
import requests
import simplejson
import unidecode
import operator
import json
import sys
from IPython.core.display import display, HTML
import pandas as pd
from collections import OrderedDict
from datetime import date
import random
from ml import *

###############PARAMETERS#######################################################
lines = ['ps4']
#example lines = ['ps4','led','tv']

facturation = 500000 # only show price * quantity > facturation
sell_condition = 'new'
#############END PARAMETERS#####################################################


pd.set_option('display.max_colwidth', -1)
display.border = 2
df_final = pd.DataFrame.from_dict([])
category = 'MLA1071'  # juegos y juguetes
categories =  { "id": "MLA5725", "name": "Accesorios para Vehículos" }, { "id": "MLA1403", "name": "Alimentos y Bebidas" }, { "id": "MLA1071", "name": "Animales y Mascotas" }, { "id": "MLA1367", "name": "Antigüedades" }, { "id": "MLA1368", "name": "Arte y Artesanías" }, { "id": "MLA1743", "name": "Autos, Motos y Otros" }, { "id": "MLA1384", "name": "Bebés" }, { "id": "MLA1246", "name": "Belleza y Cuidado Personal" }, { "id": "MLA1039", "name": "Cámaras y Accesorios" }, { "id": "MLA1051", "name": "Celulares y Teléfonos" }, { "id": "MLA1798", "name": "Coleccionables y Hobbies" }, { "id": "MLA1648", "name": "Computación" }, { "id": "MLA1144", "name": "Consolas y Videojuegos" }, { "id": "MLA1276", "name": "Deportes y Fitness" }, { "id": "MLA5726", "name": "Electrodomésticos y Aires Ac." }, { "id": "MLA1000", "name": "Electrónica, Audio y Video" }, { "id": "MLA2547", "name": "Entradas para Eventos" }, { "id": "MLA407134", "name": "Herramientas y Construcción" }, { "id": "MLA1574", "name": "Hogar, Muebles y Jardín" }, { "id": "MLA1499", "name": "Industrias y Oficinas" }, { "id": "MLA1459", "name": "Inmuebles" }, { "id": "MLA1182", "name": "Instrumentos Musicales" }, { "id": "MLA3937", "name": "Joyas y Relojes" }, { "id": "MLA1132", "name": "Juegos y Juguetes" }, { "id": "MLA3025", "name": "Libros, Revistas y Comics" }, { "id": "MLA1168", "name": "Música, Películas y Series" }, { "id": "MLA1430", "name": "Ropa y Accesorios" }, { "id": "MLA409431", "name": "Salud y Equipamiento Médico" }, { "id": "MLA1540", "name": "Servicios" }, { "id": "MLA1953", "name": "Otras categorías"}

https://api.mercadolibre.com/sites/MLA/categories
https://api.mercadolibre.com/sites/MLA/search?q=fabrica&category=MLA1403

In [ ]:


min_price_list = (1,2000,10000)#,3000,8000,16000)
max_price_list = (2000,10000,20000)#,3000,8000,16000,240000)


for num in range(0,len(list(lines))):
    product = lines[num]
    for category in categories:
        print(product," category : ",category['name']) # PRODUCT NAME SEARCHED
        df_temp = display_product(product,min_price_list,max_price_list,sell_condition,0,category['id'],facturation)
        if (len(df_temp)>0):
            df_final = df_final.append(df_temp)
        if ( len(df_final) == 0):
            df_final = df_temp
        print ("df_final length is :",len(df_final))
        df_final.to_csv('df_final.csv')

# dropping duplicates   
df_final = df_final.drop_duplicates('permalink')

In [ ]:
# show how many found
print ("Total articles found : ",len(df_final), "when price*quantity > ",facturation)

In [ ]:
# retrieve precise sold quantity ( api rounds to minimum. ex : 70000 rounds to 5000 )
# each article takes 1 second
df_final['sold_quantity'] = df_final.apply(lambda row: retrieve_sold_from_link(row['permalink']) , axis=1)
# SKIP IF PRECISE INFO IS NOT NEEDED

In [ ]:
df_final['total'] = df_final['price'] * df_final['sold_quantity']

In [ ]:
display(HTML(df_final.loc[:, ['title', 'price','sold_quantity','total','state','city','thumbnail','permalink','seller_id','free_shipping']].sort_values('total',ascending=False).to_html(border=1, escape=False ,formatters=dict(thumbnail=path_to_image_html,permalink=path_to_link_html))))
 